# Data Cleaning, Manipulation, and Analysis

In [ ]:
#importing the necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Reading the files
fact_bookings = pd.read_csv('fact_bookings.csv')
fact_agg_bookings = pd.read_csv('fact_aggregated_bookings.csv')
dim_rooms = pd.read_csv('dim_rooms.csv')
dim_hotels = pd.read_csv('dim_hotels.csv')
dim_date = pd.read_csv('dim_date.csv')


In [ ]:
#Inspecting the table
fact_bookings.head()

In [ ]:
#Inspecting the table
fact_agg_bookings.head()

In [ ]:
#Inspecting the table
dim_rooms.head()

In [ ]:
#Inspecting the table
dim_hotels.head()

In [ ]:
#Inspecting the table
dim_date.head()

In [ ]:
# Converting the string date types to date types that can be read/manipulated in python
fact_bookings['booking_date'] = pd.to_datetime(fact_bookings['booking_date'])
fact_bookings['check_in_date'] = pd.to_datetime(fact_bookings['check_in_date'])
fact_bookings['checkout_date'] = pd.to_datetime(fact_bookings['checkout_date'])
dim_date['date'] = pd.to_datetime(dim_date['date'])

In [ ]:
# Finding the number of duplicate values to be dropped out.
print(fact_bookings.duplicated().sum())
print(fact_agg_bookings.duplicated().sum())
print(dim_rooms.duplicated().sum())
print(dim_hotels.duplicated().sum())
print(dim_date.duplicated().sum())

In [ ]:
#The data files doesn't contain duplicate values.

In [ ]:
#Building a single dataset for easy access (Merging)
df = fact_bookings.merge(dim_rooms, left_on='room_category', right_on='room_id',how='left')
df = df.merge(dim_hotels, on='property_id',how='left')
df = df.merge(dim_date, left_on='check_in_date', right_on='date',how='left')

In [ ]:
#For more insights
df['stay_length'] = (df['checkout_date'] - df['check_in_date']).dt.days
df['lead_time'] = (df['check_in_date'] - df['booking_date']).dt.days
df['is_weekend'] = df['check_in_date'].dt.weekday >= 5
df['is_cancelled'] = df['booking_status'].str.lower() == 'cancelled'
df['weekday_name'] = df['check_in_date'].dt.day_name()
df['month'] = df['check_in_date'].dt.to_period('M')
df['quarter'] = df['check_in_date'].dt.quarter
df['year'] = df['check_in_date'].dt.year
df['week_number'] = df['check_in_date'].dt.isocalendar().week
df['channel'] = df['booking_platform'].str.lower()

In [ ]:
df['total_price'] = df['revenue_realized']

In [ ]:
#Making a separate summary table
summary = df.groupby('is_weekend').agg({
    'booking_id': 'count',
    'is_cancelled': 'sum',
    'total_price': 'sum',
    'stay_length': 'mean',
    'lead_time': 'mean'
}).rename(columns={
    'booking_id': 'total_bookings',
    'is_cancelled': 'total_cancellations',
    'total_price': 'total_revenue',
    'stay_length': 'avg_stay_length',
    'lead_time': 'avg_lead_time'
}).reset_index()

summary['cancellation_rate'] = summary['total_cancellations'] / summary['total_bookings']

In [ ]:
print(summary)

In [ ]:
# Bar Graph showing Total Bookings on Weekday and Weekend
sns.barplot(data=summary, x='is_weekend', y='total_bookings')
plt.title("Total Bookings: Weekday vs. Weekend")
plt.show()

In [ ]:
# Bar Graph showing cancellation rate on Weekday and Weekend
sns.barplot(data=summary, x='is_weekend', y='cancellation_rate')
plt.title("Cancellation Rate: Weekday vs. Weekend")
plt.show()

In [ ]:
# Bar Graph showing Total revenue generated on Weekday and Weekend
sns.barplot(data=summary, x='is_weekend', y='total_revenue')
plt.title("Total Revenue: Weekday vs. Weekend")
plt.show()


In [ ]:
# Bar Graph showing average stay length on Weekday and Weekend
sns.barplot(data=summary, x='is_weekend', y='avg_stay_length')
plt.title("Average Stay Length: Weekday vs. Weekend")
plt.show()


In [ ]:
# Bar Graph showing Lead Time on Weekday and Weekend
sns.barplot(data=summary, x='is_weekend', y='avg_lead_time')
plt.title("Lead Time: Weekday vs. Weekend")
plt.show()


In [ ]:
# Average WeekDay Revenue
df['adr'] = df['total_price'] / df['stay_length']
weekday_adr = df.groupby('is_weekend')['adr'].mean()
print(weekday_adr)



In [ ]:
# # Bar Graph showing average Revenue generated from Different Classes on Weekday and Weekend
df.groupby(['is_weekend', 'room_class'])['total_price'].mean().unstack().plot(kind='bar')
plt.title("Average Revenue by Room Class: Weekday vs. Weekend")
plt.show()


In [ ]:
# Estimate potential loss from cancellations
cancelled_revenue_loss = df[df['is_cancelled'] == True]['revenue_generated'].sum()
print(f"Estimated Revenue Lost Due to Cancellations: ₹{cancelled_revenue_loss:,.0f}")


# ROOT CAUSE ANALYSIS

In [ ]:
# Ensure both columns are datetime types
fact_bookings['check_in_date'] = pd.to_datetime(fact_bookings['check_in_date'])
fact_agg_bookings['check_in_date'] = pd.to_datetime(fact_agg_bookings['check_in_date'])
dim_date['date'] = pd.to_datetime(dim_date['date'])



In [ ]:
# Merging and enriching your dataset
room_perf = (
    fact_agg_bookings
    .merge(dim_rooms, left_on="room_category", right_on="room_class", how="left")
    .merge(dim_hotels, on="property_id", how="left")
    .merge(dim_date, left_on="check_in_date", right_on="date", how="left"))

In [ ]:
# To connect actual revenue to the booking performance.
booking_revenue = (
    fact_bookings
    .groupby(['property_id', 'check_in_date', 'room_category'])['revenue_realized']
    .sum()
    .reset_index())

In [ ]:
# KPIs  for evaluating performances
room_perf = room_perf.merge(
    booking_revenue,
    on=['property_id', 'check_in_date', 'room_category'],
    how='left'
)

In [ ]:
room_perf['occupancy_pct'] = room_perf['successful_bookings'] / room_perf['capacity']
room_perf['revpar'] = room_perf['revenue_realized'] / room_perf['capacity']
room_perf['revenue_per_booking'] = room_perf['revenue_realized'] / room_perf['successful_bookings']

In [ ]:
# A summary table that summarizes performance with time
room_summary = (
    room_perf
    .groupby(['property_name', 'room_category', 'day_type'])
    .agg({
        'successful_bookings': 'sum',
        'capacity': 'sum',
        'revenue_realized': 'sum'
    })
    .reset_index()
)

room_summary['occupancy_pct'] = room_summary['successful_bookings'] / room_summary['capacity']
room_summary['revpar'] = room_summary['revenue_realized'] / room_summary['capacity']
room_summary['revenue_per_booking'] = room_summary['revenue_realized'] / room_summary['successful_bookings']


In [ ]:
# sort by RevPAR to find underperformers
underperforming_rooms = room_summary.sort_values(by='revpar').head(10)



In [ ]:
#Visualizing with a scatter plot
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(
    data=room_summary,
    x='occupancy_pct',
    y='revpar',
    hue='room_category'
)
plt.axhline(0, color='red', linestyle='--')
plt.title("Room Category Performance")
plt.show()


In [ ]:
# isolating all bookings marked as "cancelled".
cancelled_df = fact_bookings[fact_bookings['booking_status'].str.lower() == 'cancelled']

In [ ]:
# Standardizing
fact_bookings['booking_status'] = fact_bookings['booking_status'].str.strip().str.lower()
fact_bookings['booking_date'] = pd.to_datetime(fact_bookings['booking_date'])
fact_bookings['check_in_date'] = pd.to_datetime(fact_bookings['check_in_date'])
fact_bookings['lead_time_days'] = (fact_bookings['check_in_date'] - fact_bookings['booking_date']).dt.days


In [ ]:
# Calculating which platforms have highest cancellation rates.
cancel_rate_by_platform = (
    fact_bookings.groupby(['booking_platform', 'booking_status'])
    .size()
    .unstack(fill_value=0))
cancel_rate_by_platform['cancel_rate'] = (
    cancel_rate_by_platform.get('cancelled', 0) /
    cancel_rate_by_platform.sum(axis=1))

In [ ]:
# Calculating which room categories have highest cancellation rates.
cancel_rate_by_room = (
    fact_bookings.groupby(['room_category', 'booking_status'])
    .size()
    .unstack(fill_value=0))
cancel_rate_by_room['cancel_rate'] = (
    cancel_rate_by_room.get('cancelled', 0) /
    cancel_rate_by_room.sum(axis=1))

In [ ]:
#Calculating weekday vs weekend cancellation rates.
fact_bookings['checkin_weekday'] = fact_bookings['check_in_date'].dt.dayofweek
fact_bookings['is_weekend_checkin'] = fact_bookings['checkin_weekday'] >= 5

In [ ]:
fact_bookings.columns
fact_bookings['booking_status'].unique()


In [ ]:
# Plotting graph for Cancellation Rate by Booking Platforms
plt.figure(figsize=(10, 5))
sns.barplot(
    data=cancel_rate_by_platform.reset_index(),
    x='booking_platform',
    y='cancel_rate',
    palette='coolwarm'
)
plt.title('Cancellation Rate by Booking Platform')
plt.ylabel('Cancellation Rate')
plt.xlabel('Booking Platform')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Plotting graph for Cancellation Rate by Room Category
plt.figure(figsize=(8, 5))
sns.barplot(
    data=cancel_rate_by_room.reset_index(),
    x='room_category',
    y='cancel_rate',
    palette='viridis'
)
plt.title('Cancellation Rate by Room Category')
plt.ylabel('Cancellation Rate')
plt.xlabel('Room Category')
plt.tight_layout()
plt.show()


In [ ]:
# Calculating total realized revenue per hotel
revenue_summary = fact_bookings.groupby('property_id')['revenue_realized'].sum().reset_index()

# Merging with hotel names
hotel_revenue = revenue_summary.merge(dim_hotels[['property_id', 'property_name']], on='property_id', how='left')

# Classifying hotels based on name
def classify_hotel(name):
    name = name.lower()
    if 'resort' in name or 'luxury' in name:
        return 'Luxury'
    elif 'express' in name or 'inn' in name or 'suites' in name:
        return 'Business'
    else:
        return 'Midscale'

hotel_revenue['hotel_category'] = hotel_revenue['property_name'].apply(classify_hotel)


In [ ]:
# Calculating Hotel Revenue
def estimate_ancillary(row):
    revenue = row['revenue_realized']
    category = row['hotel_category']
    if category == 'Luxury':
        return pd.Series({'spa_estimated': revenue * 0.05, 'fnb_estimated': revenue * 0.20, 'other_estimated': revenue * 0.02})
    elif category == 'Business':
        return pd.Series({'spa_estimated': revenue * 0.005, 'fnb_estimated': revenue * 0.25, 'other_estimated': revenue * 0.02})
    else:  # Midscale
        return pd.Series({'spa_estimated': revenue * 0.01, 'fnb_estimated': revenue * 0.15, 'other_estimated': revenue * 0.015})

hotel_revenue[['spa_estimated', 'fnb_estimated', 'other_estimated']] = hotel_revenue.apply(estimate_ancillary, axis=1)


In [ ]:
# Plotting a Pie graph for depicting share of Ancillary Revenue Components Across All Hotels

import matplotlib.pyplot as plt


total_spa = hotel_revenue['spa_estimated'].sum()
total_fnb = hotel_revenue['fnb_estimated'].sum()
total_other = hotel_revenue['other_estimated'].sum()


ancillary_totals = [total_spa, total_fnb, total_other]
labels = ['Spa', 'F&B', 'Other Services']
colors = ['#8dd3c7', '#ffffb3', '#bebada']


plt.figure(figsize=(6, 6))
plt.pie(
    ancillary_totals,
    labels=labels,
    autopct='%1.1f%%',
    colors=colors,
    startangle=140,
    wedgeprops={'edgecolor': 'black'}
)
plt.title('Share of Ancillary Revenue Components Across All Hotels')
plt.tight_layout()
plt.show()



In [ ]:
# Filtering for successful bookings
successful = fact_bookings[fact_bookings['booking_status'] == 'Checked Out'].copy()

# Merging room_class info
successful = successful.merge(dim_rooms, left_on='room_category', right_on='room_class', how='left')

# Creating 'category' from 'room_class'
successful['category'] = successful['room_class']

# Standardizing
successful['check_in_date'] = pd.to_datetime(successful['check_in_date'])
successful['week_year'] = successful['check_in_date'].dt.strftime('%Y-W%U')

# Creating weekly aggregate — count of bookings as success, and assume fixed capacity per week
agg_bookings = successful.groupby(['category', 'week_year']).agg({
    'booking_id': 'count',  # Number of bookings = demand
    'revenue_realized': 'sum'
}).rename(columns={
    'booking_id': 'successful_bookings',
    'revenue_realized': 'revenue_est'
}).reset_index()

# Estimating room capacity (assumed 10 rooms × 7 days = 70 room-nights per week)
agg_bookings['capacity'] = 70  # Assumption

agg_bookings.head()


In [ ]:
# Calculating KPIs
agg_bookings['occupancy_pct'] = (agg_bookings['successful_bookings'] / agg_bookings['capacity']) * 100
agg_bookings['revpar'] = agg_bookings['revenue_est'] / agg_bookings['capacity']
agg_bookings['adr'] = agg_bookings['revenue_est'] / agg_bookings['successful_bookings']

# Computing Week-on-Week (WoW) trends
agg_bookings.sort_values(by=['category', 'week_year'], inplace=True)

# Calculating WoW % change for each KPI
agg_bookings[['wow_occupancy', 'wow_adr', 'wow_revpar']] = agg_bookings.groupby('category')[
    ['occupancy_pct', 'adr', 'revpar']
].pct_change().fillna(0) * 100  # Convert to percentage

agg_bookings.head()


In [ ]:
df = fact_agg_bookings.merge(dim_hotels, on='property_id', how='left') \
    .merge(dim_date, left_on='check_in_date', right_on='date', how='left')


In [ ]:
# Ensuring Standarization
df['check_in_date'] = pd.to_datetime(df['check_in_date'])

# Creating 'day_type' column
df['day_type'] = df['check_in_date'].dt.dayofweek.apply(lambda x: 'Weekday' if x < 5 else 'Weekend')


In [ ]:
# Calculating Occupancy %
df['occupancy_pct'] = (df['successful_bookings'] / df['capacity']) * 100

In [ ]:
# Plotting a graph showing Occupancy %: Weekday vs Weekend
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.boxplot(x='day_type', y='occupancy_pct', data=df)
plt.title('Occupancy %: Weekday vs Weekend')
plt.ylabel('Occupancy %')
plt.xlabel('Day Type')
plt.show()


In [ ]:
# Plotting a graph showing Occupancy % by Hotel Category
plt.figure(figsize=(8, 5))
sns.boxplot(x='category', y='occupancy_pct', data=df)
plt.title('Occupancy % by Hotel Category')
plt.ylabel('Occupancy %')
plt.xlabel('Hotel Category')
plt.xticks(rotation=45)
plt.show()


In [ ]:
df['month'] = df['check_in_date'].dt.month

# Group and plotting average occupancy by month
monthly_occupancy = df.groupby('month')['occupancy_pct'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_occupancy, x='month', y='occupancy_pct', marker='o')
plt.title('Average Monthly Occupancy %')
plt.xlabel('Month')
plt.ylabel('Occupancy %')
plt.xticks(range(1, 13))
plt.grid(True)
plt.show()
